### 0. Existing Possiblities

1. Make classifications for all the existing combinations of diseases for example: d1 + d2 as d9

2. Binary classify each kind of disease

3. Make multiple models according to the data analysis 

# 2_ChestX-ray-8_The Demo Training

### 0. notes

1. The medical diagnose classification is different from MINST classification or Shopee Cloth classification, in a sense that the diagnose can be multi-classed, but the given classification examples are single-classed

### 1. import modules

In [ ]:
import numpy as np

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
import os

### 2. define the model

In [ ]:
batch_size = 32
num_classes = 8
epochs = 1
data_augmentation = False
num_predictions = 20
# The method getcwd() returns current working directory of a process.
save_dir = os.path.join(os.getcwd(), 'Demo_training_saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# The data, split between train and test sets:
y = np.load("F:/Data/Chest_Xray_Dataset/CXR-8-full-data/processed/label_array_batch_0.npy")
x = np.load("F:/Data/Chest_Xray_Dataset/CXR-8-full-data/processed/image_array_batch_0.npy")

x = np.expand_dims(x, axis=3)

y_train = y[0:100,:]
x_train = x[0:100,:]
y_test = y[100:128,:]
x_test = x[100:128,:]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

print('y_test shape:', y_test.shape)
print(y_test.shape[0], 'test samples')

In [ ]:
y[1]

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=(1024, 1024, 1)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(GlobalAveragePooling2D(data_format=None))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:


if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

In [ ]:
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])